# Imports

In [ ]:
# !pip install sed_eval loguru

In [2]:
import numpy as np
import sys
import os
#%load_ext autoreload

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#from src.utils.audio_to_spectrograms import *
from src.models.yamnet_train import *

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    path_prefix = ''
else:
    path_prefix = '..'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

I0000 00:00:1765020371.235983     109 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Using: cuda


In [3]:
# uncomment to retrain
# manual train test split (stratified)
np.random.seed(0)
data = pickle.load(open('data/processed/yamnet/spectrograms_train.pkl', 'rb'))
train_size = 0.8
train_idx = []
for label in np.unique(data['event_label']):
    choices = np.where(data['event_label'] == label)[0]
    train_idx.append(np.sort(np.random.choice(choices, size = int(np.round(len(choices)*train_size)), replace = False)))
train_idx = np.sort(np.concatenate(train_idx))
len(train_idx)

solver = Solver(epochs = 5, train_idx=train_idx, path_prefix = path_prefix, device = device)
solver.train()

Pre-computing features for 2500 files...


100%|██████████| 2500/2500 [01:10<00:00, 35.65it/s]


Epoch 1: train loss = 0.1394
Epoch 1: valildation loss = 0.1175
Epoch 2: train loss = 0.0896
Epoch 2: valildation loss = 0.0929
Epoch 3: train loss = 0.0742
Epoch 3: valildation loss = 0.0859
Epoch 4: train loss = 0.0621
Epoch 4: valildation loss = 0.0947
Epoch 5: train loss = 0.0531


2025-12-06 11:28:10.342 | INFO     | src.models.yamnet_train:train:204 - Saved model to checkpoints/yamnet_detector.pth


Epoch 5: valildation loss = 0.0983


In [4]:
test_path = Path("data") / "processed" / "yamnet" / "spectrograms_test.pkl"
test_data = pickle.load(open(test_path, "rb"))
filepaths = [os.path.join(DETECTION_TEST_PATH, file) for file in test_data['files']]
events = run_yamnet(filepaths)

2025-12-06 11:28:11.534 | INFO     | src.models.yamnet_train:run_yamnet:351 - Loading from checkpoint checkpoints/yamnet_detector.pth...
2025-12-06 11:28:12.256 | INFO     | src.models.yamnet_train:load_model:165 - Loaded model weights from checkpoints/yamnet_detector.pth


Pre-computing features for 500 files...


Testing...: 100%|██████████| 500/500 [00:02<00:00, 227.88it/s]


In [9]:
events['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': 1.92,
  'event_offset': 5.76}]

In [46]:
# check if pkl file is created
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'

gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':1}]
                    for ref_event in gt_events}

pred_event_dict = {}
for key, value in events.items():
    for value_i in range(len(value)):
        value[value_i] = value[value_i] | {
            "event_label": 1
        }
    pred_event_dict[key] = value

In [48]:
from loguru import logger
import sed_eval

detection_classes = [1]

def calculate_metrics(pred_event_dict, gt_event_dict, time_resolution=1.0, t_collar=0.25):

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(detection_classes, time_resolution=time_resolution)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(segment_based_metrics)

    return segment_based_metrics

segment_based_metrics = calculate_metrics(pred_event_dict, gt_event_dict, t_collar=0.5)

Segment based metrics
  Evaluated length                  : 4228.63 sec
  Evaluated files                   : 500 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 89.89 %
    Precision                       : 92.98 %
    Recall                          : 87.00 %
  Error rate
    Error rate (ER)                 : 0.20 
    Substitution rate               : 0.00 
    Deletion rate                   : 0.13 
    Insertion rate                  : 0.07 
  Accuracy
    Sensitivity                     : 87.00 %
    Specificity                     : 94.28 %
    Balanced accuracy               : 90.64 %
    Accuracy                        : 90.89 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 89.89 %
    Precision                       : 92.98 %
    Recall                          : 87.00 %
  Error rate
    Error rate (ER)                 : 0.20 
    Deletion rat

# Cut audio

In [6]:
from src.main import cut_events_from_audio

In [8]:
cut_events_from_audio((Path("data") / "processed" / "yamnet" / "extracted_audio"), events)

2025-12-06 11:28:27.026 | INFO     | __main__:cut_events_from_audio:5 - Created directory: data/processed/yamnet/extracted_audio
100%|██████████| 500/500 [00:04<00:00, 100.53it/s]
